In [179]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib.widgets import Cursor
from scipy.special import erf
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
import ipywidgets as widgets
from ipywidgets import Layout, HBox, VBox
from math import exp, e
from lmfit import Model, Parameters
import sys, os
import h5py

In [180]:
def create_hdf_file():
    #filename="Testerfile.csv"
    global hdf_file_name
    filename=file_name_input.value
    hdf_file_name=filename[0:-3]+"hdf5"
    with h5py.File(hdf_file_name,"a") as myfile:
    
        raw_data_group = myfile.create_group("raw data")
        main = myfile.create_group("clean data/main")
        analysis_group = myfile.create_group("/analysis") #different syntax same thing
        slices_group= myfile.create_group("slices")


        raw_data_group.create_dataset("X", data=X_raw)
        raw_data_group.create_dataset("Y", data=Y_raw)
        raw_data_group.create_dataset("Z", data=Z_raw)

        X1=main.create_dataset("X", data=X_raw, maxshape=(None), chunks=True)
        #print(X1.shape, X1.maxshape)
        main.create_dataset("Y", data=Y_raw, maxshape=(None), chunks=True)
        main.create_dataset("Z", data=Z_raw, maxshape=(None,None),chunks=True)

        #idk why i put this here, this actually goes into the export 
        #X_var[:]= X_raw
        #Y_var[:]=Y_raw
        #Z_var[:,:]=Z_raw


        #THIS won't work unless find_pump() is defined earlier....
        #raw_data_group.attrs['pump_wl'] = find_pump()

        myfile.attrs['Description']= "data of [insert compound name]"
        #myfile.attrs['history'] = "Created " + time.ctime(time.time())
        myfile.attrs['source'] = "Sfeir Lab"
        myfile.attrs['fileinfo'] = fileinfo
        #myfile.badinfo ="blah"
        #del myfile.badinfo
        #X_var.attrs['units'] = "nm"

In [181]:
#filename=input("Please enter your csv file name: ")
#filename="22BPNODIPS chloroform 600 200mW PTA fixed.csv"#22BPNODIPS chloroform 600 200mW PTA fixed.csv
    #P-1-3A-P-ex600nm-50uW-fs.csv
fileinfo=[]
def import_csv_func(b):
    filename=file_name_input.value
    rawdata = (open(filename)).readlines()
    global fileinfo#, myfile
    
    #extracting metadata
    i=0
    while i< len(rawdata):
        line = rawdata[i]
        if line.count(',')<10: #if it is not data but text
            fileinfo.append(line)
            rawdata.remove(line)
        else:
            i=i+1
    print(*fileinfo)
    
    #creating array, inverting it to get the axes right
    dataarray = (np.genfromtxt(rawdata, delimiter=",")).T
    
    #removing nans in the middle
    np.nan_to_num(dataarray, copy=False)
    
    #data = data[~np.isnan(data)]
    #data = data[~np.isnan(data).any(axis=1)]
    
    #separating the array into axes and data
    global X_raw, Y_raw, Z_raw, X_chirped, Y_chirped, Z_chirped, X,Y,Z
    X_raw=dataarray[0,1:].T 
    Y_raw=dataarray[1:,0]
    Z_raw=dataarray[1:,1:]
    
    #removing nans at the begining and end
    #a=np.isnan(Z_raw)
    #print(Z_raw)
    #mask = np.all(Z_raw, axis=0)
    #print(mask)
    #Z_raw=Z_raw[~mask]
    
    #creating copies for future data cleaning
    X_chirped=X_raw.copy() #ndarray.copy is a deep copy
    Y_chirped=Y_raw.copy()
    Z_chirped=Z_raw.copy()
    
    #basically a "pointer" to whicher dataset we are currently using
    X=X_raw
    Y=Y_raw
    Z=Z_raw
    
    create_hdf_file()
    #stack.selected_index = 1
    #this is for potentially making a stack out of the import and main widget
    

In [182]:
def import_hdf_func(b):
    global X_raw, Y_raw, Z_raw, X_chirped, Y_chirped, Z_chirped, X,Y,Z, fileinfo, hdf_file_name
    hdf_file_name=file_name_input.value
    with h5py.File(hdf_file_name,"a") as myfile:
        #put in some error handling if wrong hdf5 file
        if "raw data" not in myfile.keys():
            return "ERROR"

        X_raw= myfile['/raw data/X'][:]
        Y_raw= myfile['/raw data/Y'][:]
        Z_raw= myfile['/raw data/Z'][:]
        X_chirped= myfile['/clean data/main/X'][:]
        Y_chirped= myfile['/clean data/main/Y'][:]
        Z_chirped= myfile['/clean data/main/Z'][:]

        fileinfo=myfile.attrs['fileinfo']
        #analysis_grp= myfile['analysis']
        #basically a "pointer" to whicher dataset we are currently using
        X=X_raw
        Y=Y_raw
        Z=Z_raw
    print(*fileinfo)
    #print("done w import")
        #stack.selected_index = 1
        #copy over + tweak the importer
        #save the metadata as a variable, dont automatically stick it in a file


In [183]:
#----setup for importing-------
file_name_input = widgets.Text(
    value='22BPNODIPS chloroform 600 200mW PTA fixed.csv',
    placeholder='myfile.csv',
    description='Filename:',
)
import_hdf_button= widgets.Button(
    description='Import existing dataset from HDF',
    layout=Layout(width="50%"))

import_csv_button= widgets.Button(
    description='Create new dataset from CSV',
    layout=import_hdf_button.layout)

importing = VBox([file_name_input, HBox([import_hdf_button, import_csv_button])])

#the Stacked widget would be perfect and elegant but it is not in the update of ipywidgets
#stack = widgets.Stacked([importing, tab])
#not_stack= VBox([importing,tab])

import_hdf_button.on_click(import_hdf_func)
import_csv_button.on_click(import_csv_func)

importing


 Date: March 16, 2015
 Sample: NODIPS-22BP-Chloroform
 Solvent: THF
 Pump energy (uJ): 200 uW
 Pump wavelength (nm): 600
 Cuvette length (mm): 2
 Comments: Time Zero: 505.800 ps
 Averaging time: 2.0 s
 Number of scans: 2
 Measurement time: 00:37:49
 
 Time units: ps
 Z axis title: dA


In [192]:
#-----Save Options Tab setup------
save_hdf_dropdown= widgets.Dropdown(
    options=[(" ",0),("save current general plot as main",1), ("save current general plot as alternate",2), ("save current vertical slice",3), ("save current horizontal slice",4), ("save SVD data",5)],
    description='HDF Export Options',
    style = {'description_width': 'initial'}) 
#^for some of thsoe options i might need to ask for a filename? eh that's a stretch goal currently let's name things the standard way
export_csv_dropdown= widgets.Dropdown(
    description='CSV Export Options',
    options=[(" ",0),("export current clean data",1), ("export analyses",2), ("export all slices of interest",3)],
    style = {'description_width': 'initial'}) 

save_button = widgets.Button(description='save')
export_button = widgets.Button(description='Export')

save_options=VBox([HBox([save_hdf_dropdown,save_button]),HBox([export_csv_dropdown,export_button])])

def save_func(b):
    with h5py.File(hdf_file_name,"a") as myfile:
        val= save_hdf_dropdown.value 
        if val==0:
            print("please select a value!")
            return
        if val==1:
            X_var=myfile["/clean data/main/X"]
            if X_var.size != X_chirped.size :
                #print("dset size: ",X_var.size, "array size:",X_chirped.size,"dset maxsize: ", X_var.maxshape)
                X_var.resize(X_chirped.shape)
                #print("new dset size: ", X_var.size)
                myfile["/clean data/main/Z"].resize(Z_chirped.shape)
            X_var[:]= X_chirped
            myfile["/clean data/main/Y"][:]= Y_chirped
            myfile["/clean data/main/Z"][:]= Z_chirped
            return
        if val==2:
            i=1
            while f"alt_{i}" in myfile["/clean data"].keys():
                i+=1
            #pretty sure this syntax works
            myfile.create_group(f"/clean data/alt_{i}")
            myfile[f"/clean data/alt_{i}/X"]= X_chirped
            myfile[f"/clean data/alt_{i}/Y"]= Y_chirped
            myfile[f"/clean data/alt_{i}/Z"]= Z_chirped
            return
        if val==3:
            if not ax2.get_lines():
                print("you don't have a slice selected. click anywhere on the graph to select a slice")
                return 
            wl= ax2.get_title().split()[-1]
            Y=ax2.get_lines()[0].get_xdata()
            Z=ax2.get_lines()[0].get_ydata()
            if f"v_{wl}" in myfile["/slices"].keys():
                print("this slice already exists, no changes made")
            else: 
                myfile[f"/slices/v_{wl}/Y"]= Y
                myfile[f"/slices/v_{wl}/Z"]= Z
            return
        if val==4:
            if not ax2.get_lines():
                print("you don't have a slice selected. click anywhere on the graph to select a slice")
                return
            t= ax3.get_title().split()[-1]
            X=ax3.get_lines()[0].get_xdata()
            Z=ax3.get_lines()[0].get_ydata()
            if f"h_{t}" in myfile["/slices"].keys():
                print("this slice already exists, no changes made")
            else: 
                myfile[f"/slices/h_{t}/X"]= X
                myfile[f"/slices/h_{t}/Z"]= Z
            return
        if val==5: 
            #save svd data
            print("this function coming soon")
            return



def export_func(b):
    with h5py.File(hdf_file_name,"a") as myfile:
        val= export_csv_dropdown.value 
        if val==0:
            print("please select a value!")
            return
        if val==1: #clean data
            filename=file_name_input.value
            csv_file_name=filename.split(".")[0]+" cleaned.csv"

            step1 =np.insert(Z_chirped,0,Y_chirped, axis=1)
            step2= np.insert(X_chirped,0,0,axis=0)
            step3= np.insert(step1,0,step2,axis=0)
            np.savetxt(csv_file_name, step3, delimiter=",")
            return
        if val==2: #analyses
            print("coming soon!")
            return
        if val==3: #slices of interest
            if not os.path.exists("slices"):
                os.mkdir("slices")
            for key in myfile["/slices"].keys():
                if key[0]=="v": #y,z
                    combined_yandz=np.vstack((myfile["/slices"][key + "/Y"][:],myfile["/slices"][key + "/Z"][:]))
                    np.savetxt("slices/"+key+".csv", combined_yandz, delimiter=",")

                if key[0]=="h":# x,z
                    combined_xandz= np.vstack((myfile["/slices"][key + "/X"][:],myfile["/slices"][key + "/Z"][:]))
                    np.savetxt("slices/"+key+".csv", combined_xandz, delimiter=",")
            #print("please select a value!")
            return
    

save_button.on_click(save_func)    
export_button.on_click(export_func)

In [193]:
#----noise removal functions----

#uses global metadata (called fileinfo) to find the pump wavelength
def find_pump():
    #fileinforead=open(fileinfo)
    #global myfile
    
    pumpwl=0
    for line in fileinfo:
        if line[0:15]=='Pump wavelength':
            pumpwl= int(line[-4:-1])
            #print(pumpwl)
    if pumpwl!=0:
        with h5py.File(hdf_file_name,"a") as myfile:
            myfile.attrs['pumpwl'] = pumpwl
        return [pumpwl-20, pumpwl+20]
    else:
        print("did not find data")
        return [0,0]
    
#---old functions not being used------
def remove_pump():
    #step 4: remove the laser
    #add a manual option
    [start,end]= find_pump()
    if start!=0:
        starti = np.searchsorted(X, start)#this can be adjusted
        endi = np.searchsorted(X, end)
        Z[:,starti:endi]=0
    else:
        print("did not find data")
    Z_chirped=Z
    
def remove_fringe_noise():
    global X,Z, X_chirped, Z_chirped
    listofR=[]
    #print((np.shape(X)))
    
    coef=0.4 #for first dataset it was 0.4, second 0.2 THIS NEEDS ADJUSTING
    for col in range((np.shape(X))[0]):
        slice2=Z[:,col]
        R=abs(np.corrcoef(Y,slice2))[0,1]
        listofR.append(R)
        #print(R)
    i=0
    while listofR[i]<coef: #find uncorrelated data in the beginning 
        Z=np.delete(Z,0,1) #remove that row from Z
        X=np.delete(X,0,0) #remove the wavelength too
        i=i+1
    i=(np.shape(X))[0]-1#same thing but from the end
    while listofR[i]<coef:
        Z=np.delete(Z,-1,1)
        X=np.delete(X,-1,0)
        i=i-1
    X_chirped=X
    Z_chirped=Z
#---end of old functions not being used
    
def remove_pump2(values):
    global Z_chirped, Z,X
    starti = np.searchsorted(X, values[0])
    endi = np.searchsorted(X, values[1])
    Z[:,starti:endi]=0
    Z_chirped=Z
        
def remove_fringe_noise2(values):
    global Z_chirped, X_chirped, Z,X
    start= values[0]
    end=values[1]
    while X[0]<start: 
        Z=np.delete(Z,0,1)
        X=np.delete(X,0,0) 
    while X[-1]>end:
        Z=np.delete(Z,-1,1)
        X=np.delete(X,-1,0)
    X_chirped=X
    Z_chirped=Z

pump_range=widgets.IntRangeSlider(
    value=[find_pump()[0], find_pump()[1]],
    min=X_raw[0],
    max=X_raw[-1],
    step=1,
    description='Select the pump range to be eliminated:',
    continuous_update=False,
    readout=True,
    readout_format='d',
    style={'description_width' :'initial'},
    layout=Layout(width="80%"))
edge_range= widgets.IntRangeSlider(
    value=[X[0],X[-1]],
    min=X_raw[0],
    max=X_raw[-1],
    step=1,
    description='Select the bounds of the noise at the edges:',
    continuous_update=False,
    readout=True,
    readout_format='d',
    style={'description_width' :'initial'},
    layout=Layout(width="80%"))
                
cleaning_button = widgets.Button(
    description='Click to clean',
)

clear_button = widgets.Button(
    description='Clear changes',
    disabled= True
)
    
def cleaning_func(button):
    remove_fringe_noise2(edge_range.value)
    remove_pump2(pump_range.value)
    cleaning_button.disabled = True
    clear_button.disabled= False
    #tab.selected_index=0
    tab.selected_index=1
    
def clear_func(button):
    global X_chirped, Z_chirped
    X_chirped=X_raw.copy() #ndarray.copy is a deep copy
    Z_chirped=Z_raw.copy()
    cleaning_button.disabled = False
    clear_button.disabled= True
    #tab.selected_index=1
    tab.selected_index=0
    
clear_button.on_click(clear_func)
cleaning_button.on_click(cleaning_func)

cleaning_controls=VBox([pump_range, edge_range, cleaning_button])

In [199]:
#all the chirping functions


#finds the important wavelengths for the gaussian, discarding ones where the max value of the slice is <5% 
#n=10

def find_max_slice_indices(n):
    maxZ=np.amax(Z)
    fiveper = maxZ*0.05
    x=[0]#array of indices of even chunks
    maxes=[] #temporary array of max values of each wl in the chunk
    c=(X[-1]-X[0])/n #how big the increments are

    xmaxes=[]#array of wls for which to base gaussian
    for i in range(1,n+1):
        x.append(np.searchsorted(X,(X[0]+i*c))) #end index

        for wl in range(x[i-1],x[i]):
            maxes.append(np.amax(Z[:,wl]))#finds max of each of 50 wls, appends to maxes[]
        if np.amax(maxes)< fiveper:
            maxes=[] #need to clear maxes
            continue
        else:
            wl= maxes.index(np.amax(maxes))+x[i-1] #finds index of max(maxes), adds to start of chunk
            xmaxes.append(wl)
        maxes=[] #need to clear maxes 
    print(xmaxes)
    return xmaxes

def curve3(t, y0,pw,t0,a1,k1,a2,k2,a3,k3):
    return y0+0.5*a1*(e**(-k1*(t-t0-(pw**2)*k1/2)))*(1+erf((t-t0-k1*(pw**2))/(1.41421356237*pw)))+0.5*(a2*e**(-k2*(t-t0-(pw**2)*k2/2)))*(1+erf((t-t0-k2*(pw**2))/(1.41421356237*pw)))+0.5*(a3*e**(-k3*(t-t0-(pw**2)*k3/2)))*(1+erf((t-t0-k3*(pw**2))/(1.41421356237*pw)))
def curve4(t, y0,pw,t0,a1,k1,a2,k2,a3,k3,a4,k4):#,a5,k5,a6,k6):
    return y0+0.5*a1*(e**(-k1*(t-t0-(pw**2)*k1/2)))*(1+erf((t-t0-k1*(pw**2))/(1.41421356237*pw)))+0.5*(a2*e**(-k2*(t-t0-(pw**2)*k2/2)))*(1+erf((t-t0-k2*(pw**2))/(1.41421356237*pw)))+0.5*(a3*e**(-k3*(t-t0-(pw**2)*k3/2)))*(1+erf((t-t0-k3*(pw**2))/(1.41421356237*pw)))+0.5*a4*(e**(-k4*(t-t0-(pw**2)*k4/2)))*(1+erf((t-t0-k4*(pw**2))/(1.41421356237*pw)))
def curve5(t, y0,pw,t0,a1,k1,a2,k2,a3,k3,a4,k4,a5,k5):
    return y0+0.5*a1*(e**(-k1*(t-t0-(pw**2)*k1/2)))*(1+erf((t-t0-k1*(pw**2))/(1.41421356237*pw)))+0.5*(a2*e**(-k2*(t-t0-(pw**2)*k2/2)))*(1+erf((t-t0-k2*(pw**2))/(1.41421356237*pw)))+0.5*(a3*e**(-k3*(t-t0-(pw**2)*k3/2)))*(1+erf((t-t0-k3*(pw**2))/(1.41421356237*pw)))+0.5*a4*(e**(-k4*(t-t0-(pw**2)*k4/2)))*(1+erf((t-t0-k4*(pw**2))/(1.41421356237*pw)))+0.5*a5*(e**(-k5*(t-t0-(pw**2)*k5/2)))*(1+erf((t-t0-k5*(pw**2))/(1.41421356237*pw)))
def curve6(t, y0,pw,t0,a1,k1,a2,k2,a3,k3,a4,k4,a5,k5,a6,k6):
    return y0+0.5*a1*(e**(-k1*(t-t0-(pw**2)*k1/2)))*(1+erf((t-t0-k1*(pw**2))/(1.41421356237*pw)))+0.5*(a2*e**(-k2*(t-t0-(pw**2)*k2/2)))*(1+erf((t-t0-k2*(pw**2))/(1.41421356237*pw)))+0.5*(a3*e**(-k3*(t-t0-(pw**2)*k3/2)))*(1+erf((t-t0-k3*(pw**2))/(1.41421356237*pw)))+0.5*a4*(e**(-k4*(t-t0-(pw**2)*k4/2)))*(1+erf((t-t0-k4*(pw**2))/(1.41421356237*pw)))+0.5*a5*(e**(-k5*(t-t0-(pw**2)*k5/2)))*(1+erf((t-t0-k5*(pw**2))/(1.41421356237*pw)))+0.5*pow(a6,(-k6*(t-t0-pow(pw,2)*k6/2)))*(1+erf((t-t0-k6*pow(pw,2))/(1.41421356237*pw)))

curve3model=Model(curve3)
curve4model=Model(curve4)
curve5model=Model(curve5)


def findT0(params,Y,Z,guess):#,pwguess):
    #-----The bad old way----
    #params, pc = curve_fit(curve1, Y,Z,maxfev=1000000, p0=[1e-6,0.1,guess,1e-3,1,1e-3,1e-2,1e-3,1e-3,1e-4,1e-3])
    #print(params[1], params[2], X[wl])
    ##pwguess[0]=params[1]
    #return params[2]
    
    #------the good new way-----
    #params['pw'].value= pwguess
    params['t0'].value= guess
    params['t0'].max= guess+0.2
    params['t0'].min= guess-0.2
    
    lm_output= curvemodel.fit(Z,params, t=Y, nan_policy='propagate')
    bestvals= lm_output.best_values
    return bestvals['t0'] 

#print(T0)
#pwguess=[0.1]
def find_t0s(xmaxes, left_guess, curve_num):
    #gaussian
    #left_guess=-0.4 #ADJUST -0.4,-0.2
    #curvemodel=curve3model
    global curvemodel
    curvemodel=globals()["curve"+str(curve_num)+"model"]
    #print('parameter names: {}'.format(curvemodel.param_names))
    #print('independent variables: {}'.format(curvemodel.independent_vars))

    params = curvemodel.make_params(y0=1e-6,pw=0.1,t0=left_guess,a1=1e-3,k1=1,a2=1e-3,k2=1e-2,a3=1e-3,k3=1e-3)#,a4=1e-4,k4=1e-3)#,a5=1e-5,k5=1e-3)

    params['k1'].min=0
    params['k2'].min=0
    params['k3'].min=0
    #params['k4'].min=0
    #params['k5'].min=0
    #return params
    
    #pumpwl=myfile.attrs["pumpwl"]
    pumpwl=600
    print( pumpwl)
    T0=[left_guess]
    
    for i,wl in enumerate(xmaxes):# range(0,np.shape(X)[0]): #
        #the indices around the pump wavelength that are all 0
        if (pumpwl-20)<=wl<=(pumpwl+20): 
            T0.append(0)
            continue
        #NOT USING THIS--
        #interpolate the vertical slice over more data points
        f=interp1d(Y,Z[:,wl], kind="linear", fill_value="extrapolate")   
        Ybig=np.linspace(-100,2600,20000) #start,end,num of points
        Zbig=f(Ybig)
        #------ 

        T0.append(findT0(params,Y,Z[:,wl],T0[-1]))#,pwguess[0])) 
        #print(T0[-1],X[wl])
        print(i, end =" ")
    print("done!")
    return T0

def chirp_curve(maxwls, a, b, c,d):
    return a*pow(maxwls,3)+b*pow(maxwls,2)+c*maxwls+d

def make_t0_curve(T0,xmaxes):
    maxwls=[X[0]] 
    for i in xmaxes: #converts list of indices to list of wls
        maxwls.append(X[i])

    params1, pc = curve_fit(chirp_curve, maxwls, T0, maxfev=6000)
    #params1, pc = curve_fit(chirp_curve, X, T0[:-1], maxfev=6000)
    t0_curve=chirp_curve(X, params1[0], params1[1], params1[2],params1[3])
    return t0_curve, maxwls

#CHIRP CORRECTOR-------
def chirp_corrector(t0_curve):
    it0=[]
    for wl in range(0,np.shape(X)[0]):
        it0.append(np.searchsorted(Y,t0_curve[wl]))#gets the index of that wl's t0

    #T0=float(y[0])#-0.6
    iT0=np.searchsorted(Y,chirp_calculations.T0[0]) #index of first time zero
    Znew=Z.copy()
    Ynew=Y.copy()
    for wl in range(0,np.shape(X)[0]):
        shift=0-t0_curve[wl] #note, not all data is around zero, might have to change
        for t in range(iT0,np.shape(Y)[0]-1):#start changing from main T0
            Ynew[t]=Y[t]+shift
            Znew[t,wl]=Z[t,wl]
        f=interp1d(Ynew,Znew[:,wl], kind="linear", fill_value="extrapolate")       
        Znew[:,wl]=f(Y)
    return Znew

def final_chirping(b):
    global Z_chirped
    Z_chirped= chirp_corrector(chirp_calculations.t0_curve)
    plot_main_plot({'new':1})
    ax.set_ylim(-1,1)
    
def clear_chirp_curves(b):
    lines=ax.get_lines()
    lines[-1].remove
    lines[-1].remove
    
def get_chirp_inputs():
    return (int(n.value),float(left_guess.value),int(curve_num.value))

def chirp_calculations(b):
    n, left_guess,curve_num= get_chirp_inputs()
    xmaxes= find_max_slice_indices(n)
    #params= make_curve_params(left_guess, curve_num)
    chirp_calculations.T0= find_t0s(xmaxes, left_guess, curve_num)
    chirp_calculations.t0_curve, maxwls= make_t0_curve(chirp_calculations.T0,xmaxes)
    
    ax.plot(maxwls,chirp_calculations.T0, color="yellow")
    ax.plot(X,chirp_calculations.t0_curve, color="white")
    #return chirp_calculations.t0_curve
    yes_btn = widgets.Button(description="yes go for it")
    no_btn = widgets.Button(description="no, let me put different numbers")
    with chirp_output:
        display(yes_btn, no_btn)
        yes_btn.on_click(final_chirping)
        no_btn.on_click(clear_chirp_curves)
    return chirp_calculations.t0_curve, chirp_calculations.T0
    
def chirp_wrapper(b):
    #this should cause the thing to appear
    ax.set_ylim(-1,1)
    n = widgets.Text(description="# of points to base the curve on?", value= "15")
    left_guess = widgets.Text(description="leftmost time zero", value = "-0.4")
    curve_num = widgets.Text(description="# of curves to use", value="3")
    chirp_processor_button = widgets.Button(description="do the math!")
    with chirp_output:
        display(n, left_guess, curve_num, chirp_processor_button)
        chirp_processor_button.on_click(chirp_calculations)
        #n=int(input("how many points do you want the curve to be based on? "))
        #left_guess=float(input("what is the leftmost t0?"))
        #curve_num=int(input("how many exponentials to use? (3)"))

#Znew=chirp_wrapper()

In [204]:
%matplotlib widget
#setup for all the widgets

# two cursors setup, referenced in onclick_cursor()
two_cursors=widgets.Checkbox(value=True, description="Two Cursors")  

chirp_start_button = widgets.Button(description="let's chirp correct")
chirp_start_button.on_click(chirp_wrapper)
chirp_output = widgets.Output()
chirp_box = VBox([chirp_start_button, chirp_output])

#horizontal slice widget
plot_hslice_output= widgets.Output()

@plot_hslice_output.capture()#clear_output = True, wait= True)
def onclick_hslice(event):
    t=event.ydata
    #get horizontal slice
    j = np.searchsorted(Y, t)#finds closest index to that t
    slice1=Z[j,:]
    [l.remove() for l in ax3.lines]
    ax3.plot(X,slice1,color="black") 
    ax3.set_title(f"Horizontal slice at time {t}")
#vertical slice widget
plot_vslice_output= widgets.Output()

@plot_vslice_output.capture()#clear_output = True, wait= True)
def onclick_vslice(event):
    wl=event.xdata
    #get vertical slice
    i = np.searchsorted(X, wl)#finds closest index to that wl
    slice1=Z[:,i]
    [l.remove() for l in ax2.lines] #->this needs to change to keep last plot
    ax2.plot(Y,slice1,color="black")
    ax2.set_title(f"Vertical slice at wavelength {wl}")
    


In [205]:
#---Main Plot Setup----
%matplotlib widget
a=widgets.Checkbox(description="a checkbox")
      
def which_X_to_plot(tab_value):   #David approved
    global X,Y,Z
    if tab_value==0:
        X=X_raw
        Y=Y_raw
        Z=Z_raw
    if tab_value==1:
        X=X_chirped
        Y=Y_chirped
        Z=Z_chirped
        

plot_output=widgets.Output()

#Main Plot function: (called at start, and when switching tabs)
def plot_main_plot(change):
    
    #important, switches datasets when switching tabs
    tab_value=change['new']
    which_X_to_plot(tab_value) 
    
    with plot_output:
        ax.clear()
        ax.pcolormesh(X,Y,Z, cmap='jet', shading='nearest')
        #shading='nearest' makes the pixels centered on the coordinates
        plt.ylabel('Time(ps)')
        plt.xlabel('Wavelength(nm)')
        #plt.ylim(-1,1)
        #plt.yscale('log')
        #cursor=Cursor(ax, useblit=True) #!!if commenting this out, in onclick_cursor swich the range start from 2 to 0 and vice versa
    
    
@plot_output.capture()#clear_output = True, wait= True)
def onclick_cursor(event):
        #for line in ax.get_lines(): # this will clear the other cursor....
        #    line.remove()
        
        #is better cause you don't need to redraw the old lines
        if two_cursors.value: keepLastLine=2
        else: keepLastLine=0
        
        lines= ax.get_lines()
        for i in range(0,len(lines)-keepLastLine): 
            lines[i].remove()
        
        pos.append([event.xdata,event.ydata])
        cursorx=ax.axvline(x=pos[-1][0], zorder=5)
        cursory=ax.axhline(y=pos[-1][1], zorder=5)
        #cursorx=ax.axvline(x=event.xdata, zorder=5)
        #cursory=ax.axhline(y=event.ydata, zorder=5)


In [206]:
#---Tab Structure Setup---
slices_tabs=widgets.Tab()
slices_tabs.children = [plot_hslice_output, plot_vslice_output]
slices_tabs.set_title(0, 'Horizontal')
slices_tabs.set_title(1, 'Vertical')

raw_data_box = VBox([two_cursors, plot_output, pump_range, edge_range, cleaning_button, slices_tabs])
clean_data_box = VBox([two_cursors, chirp_box, plot_output, clear_button, slices_tabs])
SVD = a
#save_options = VBox([save_hdf_dropdown,export_csv_dropdown]) #defined up there

tab = widgets.Tab()
tab.children = [raw_data_box, clean_data_box, SVD, save_options]
tab.set_title(0, 'Raw Data')
tab.set_title(1, 'General')
tab.set_title(2, 'SVD')
tab.set_title(3, 'Save Options')
#tab.observe(which_X_to_use1, names='selected_index')
tab.observe(plot_main_plot, names='selected_index')

#---creating the figures in the output---

with plot_output:
    fig, ax = plt.subplots()
with plot_hslice_output:
    fig3, ax3 = plt.subplots()
with plot_vslice_output:
    fig2, ax2 = plt.subplots()

plot_main_plot({'new':0})
pos = []
with plot_output:
    fig.canvas.mpl_connect('button_press_event', onclick_cursor)
    fig.canvas.mpl_connect('button_press_event', onclick_hslice)
    fig.canvas.mpl_connect('button_press_event', onclick_vslice)



In [207]:
tab

In [ ]:
#----FOR TESTING-----
%matplotlib widget
fig=plt.figure()
wl=480
i=np.searchsorted(X,wl)
#params['t0'].value=
one_output= curvemodel.fit(Z[:,i],params, t=Y, nan_policy='propagate')
bestvals1= one_output.best_values
print(bestvals1['t0'])
curveplot=one_output.plot(fig=fig) #plots fit and resids. use plot_fit() or plot_residuals() for each
plt.xlim(-1,2)


In [ ]:
#---For testing but with slider----
%matplotlib widget
fig, ax = plt.subplots()


def slider(wl):
    #finds closest index to that t
    i=np.searchsorted(X,wl)
    #get horizontal slice
    plt.clf()
    
    one_output= curvemodel.fit(Z[:,i],params, t=Y, nan_policy='propagate')
    one_output.plot(fig=fig) #plots fit and resids. use plot_fit() or plot_residuals() for each
    plt.xlim(-1,2)
    
    
widgets.interact(slider, wl=(400,800,2))


In [ ]:
#plotter of chirp before and after------
%matplotlib widget
fig, (ax1, ax2) = plt.subplots(1, 2)

ax1.pcolormesh(X,Y,Z, cmap='jet', shading='nearest') #before
ax1.axis(ymin=-1,ymax=2)
ax2.pcolormesh(X,Y,Znew, cmap='jet', shading='nearest')#manual curve
ax2.axis(ymin=-1,ymax=2)


In [ ]:
class SVD_data:
    def __init__(self, X,Y,Z):
        self.X = X
        self.Y = Y
        self.Z = Z
        self.u, self.s, self.vh = np.linalg.svd(Z) 
        self.rankdata ={}
    
    def get_rankdata(self,*list_of_ranks):
        for numRank in list_of_ranks:
            if not (numRank in self.rankdata.keys()): 
                self.rankdata[numRank]= Rank(numRank,self)
        return self.rankdata[numRank]
    
    def plot_s(self,numranks):
        x_axis=list(range(1, numranks+1))
        plt.scatter(x_axis,self.s[:numranks])
        plt.yscale('log')
        plt.title('s')
        
    
    def plot_u_and_vh(self,*args):
        working=True
        for numRank in args:
            if not (numRank in self.rankdata.keys()): 
                print(f"ERROR: rank={numRank} is not defined. Please call SVD_data.get_rankdata(rank) to define it.")
                working = False
        if not working: sys.exit();
        if (args):
            self.rankdata[args[0]].plot_u_and_vh()
        elif (not hasattr(self, 'goodrank')):
            print("ERROR: you have not specified the rank(int) you want to plot the u and vh for as an argument. This function can also be called without arguments if you set the preferred rank with SVD_Data.set_good_rank(ranknum) ")            
        else:
            self.goodrank.plot_u_and_vh()
            
        
    def set_good_rank(self,rank):
        self.goodrank=self.get_rankdata(rank) 
    
    def plot_ddA(self, *args):
        working=True
        for numRank in args:
            if not (numRank in self.rankdata.keys()): 
                print(f"ERROR: rank={numRank} is not defined. Please call SVD_data.get_rankdata(rank) to define it.")
                working = False
        if not working: sys.exit();
        if (args):
            self.rankdata[args[0]].plot_ddA()
        elif (not hasattr(self, 'goodrank')):
            print("ERROR: you have not specified the rank(int) you want to plot the ddA for as an argument. This function can also be called without arguments if you set the preferred rank with SVD_Data.set_good_rank(ranknum) ")            
        else:
            self.goodrank.plot_ddA()
            
    def plot_reconstr(self, *args):
        working=True
        for numRank in args:
            if not (numRank in self.rankdata.keys()): 
                print(f"ERROR: rank={numRank} is not defined. Please call SVD_data.get_rankdata(rank) to define it.")
                working = False
        if not working: sys.exit();
        if (args):
            self.rankdata[args[0]].plot_reconstr()
        elif (not hasattr(self, 'goodrank')):
            print("ERROR: you have not specified the rank(int) you want to plot the reconstruction for as an argument. This function can also be called without arguments if you set the preferred rank with SVD_Data.set_good_rank(ranknum) ")            
        else:
            self.goodrank.plot_reconstr()
            
class Rank:
    def __init__(r, rank,svd_data): #want to do things without all these other vars
        r.rank = rank
        
        #u, s, vh = np.linalg.svd(svd_data.Z)
        r.u_ranked= svd_data.u[:,:rank]
        r.vh_ranked = np.transpose(svd_data.vh[:rank:,])
        s_ranked=np.diag(svd_data.s[:rank])
        
        #creating reconstruced dataset
        step1=np.matmul(r.u_ranked,s_ranked)
        step2=np.transpose(r.vh_ranked)
        r.reconstructed=np.matmul(step1,step2)
        
        r.ddA=svd_data.Z-r.reconstructed
        
    def plot_u_and_vh(r):
        fig, (ax1, ax2) = plt.subplots(1, 2)

        ax1.plot(Y,r.u_ranked)
        ax1.set_xlabel('Time(ps)')
        ax1.set_ylabel('U')


        ax2.plot(X,r.vh_ranked)
        ax2.set_xlabel('Wavelength(nm)')
        ax2.set_ylabel('Vh')
        
    def plot_reconstr(r):
        plt.pcolormesh(X,Y,r.reconstructed, cmap='jet', shading='nearest')
        plt.colorbar()
        plt.ylabel('Time(ps)')
        plt.xlabel('Wavelength(nm)')
        plt.title(f'Reconstructed dataset from SVD for rank={r.rank}')
        #probably want to return this as an object...
        #plt.ylim(-0.1,2.5)
        
    def plot_ddA(r): #must deal with getting the X and Y
        plt.pcolormesh(X,Y,r.ddA, cmap='jet', shading='nearest')
        plt.colorbar()
        plt.ylabel('Time(ps)')
        plt.xlabel('Wavelength(nm)')
        plt.title(f'Delta (Delta A) for rank={r.rank}')
        #plt.ylim(-0.1,2.5)


In [ ]:
%matplotlib widget

test_dataset=SVD_data(X,Y,Znew)

test_dataset.get_rankdata(1,2,3)
#test_dataset.set_good_rank(3) 
test_dataset.plot_u_and_vh(3)
#test_dataset.plot_s(10)

#rank_two=test_dataset.get_rankdata(4)
#test_dataset.plot_reconstr(2)
plt.ylim(-0.1,2.5)
plt.show()

In [ ]:
#testing how variables and objects in python work
from sys import getrefcount

print(getrefcount(s))
asdf=0
print(getrefcount(s))
asdf=s[3:5]
print(getrefcount(s))

#trying to make a template so that those two functions don't take up as much space 
#but i think that is a pointless endeavor
def plot_temp(self, func, *args):
        working=True
        for numRank in args:
            if not (numRank in self.rankdata.keys()): 
                print(f"ERROR: rank={numRank} is not defined. Please call SVD_data.get_rankdata(rank) to define it.")
                working = False
        if not working: sys.exit();
        if (args):
            self.rankdata[args[0]].func()
        elif (not hasattr(self, 'goodrank')):
            print("ERROR: you have not specified the rank(int) you want to plot the ddA for as an argument. This function can also be called without arguments if you set the preferred rank with SVD_Data.set_good_rank(ranknum) ")            
        else:
            self.goodrank.func()
            
def plot_reconstr(self,*args):
    self.plot_temp(Rank.plot_reconstr(), args)

In [ ]:
#--finds SVD, plots first ten values of s on log scale----
%matplotlib widget
u,s,vh = np.linalg.svd(Znew)
#print("u= ", u,"s= ",s,"vh= ",vh)
print(np.shape(u)) #time
print(np.shape(vh)) #spectra
ten=list(range(1, 11))
plt.scatter(ten,s[:10])
plt.yscale('log')
plt.title('s')

#---plots u vs wl and vh vs time
fig, (ax1, ax2) = plt.subplots(1, 2)

rank =4
u_ranked= u[:,:rank]
vh_ranked = np.transpose(vh[:rank:,])

ax1.plot(Y,u_ranked)
#ax1.plot(Y,u_ranked[:,0], label="1")
#ax1.plot(Y,u_ranked[:,1], label="2")
#ax1.plot(Y,u_ranked[:,2], label="3")
#ax1.legend()
ax1.set_xlabel('Time(ps)')
ax1.set_ylabel('U')


ax2.plot(X,vh_ranked)
#ax2.plot(X,vh_ranked[:,0], label="1")
#ax2.plot(X,vh_ranked[:,1], label="2")
#ax2.plot(X,vh_ranked[:,2], label="3")
#ax2.legend()
ax2.set_xlabel('Wavelength(nm)')
ax2.set_ylabel('Vh')

#---Reconstructing the dataset from SVD---

s_ranked=np.diag(s[:rank])
#print(s_ranked)
step1=np.matmul(u_ranked,s_ranked)
#print(np.shape(step1))
step2=np.transpose(vh_ranked)
#print(np.shape(step2))
step3=np.matmul(step1,step2)
#print(np.shape(step3))
plt.pcolormesh(X,Y,step3, cmap='jet', shading='nearest')
plt.colorbar()
plt.ylabel('Time(ps)')
plt.xlabel('Wavelength(nm)')
plt.title('Reconstructed dataset from SVD')
#plt.ylim(-0.1,2.5)
#plt.xlim(406,450)

#---plotting the residual-----
%matplotlib widget
ddA= Znew-step3
print(np.shape(ddA))
plt.pcolormesh(X,Y,ddA, cmap='jet', shading='nearest')
plt.colorbar()
plt.ylabel('Time(ps)')
plt.xlabel('Wavelength(nm)')
plt.title('Delta (Delta A)')
#plt.ylim(-0.1,2.5)
#plt.xlim(406,450)


In [ ]:
print(Znew.mean())# rank=1 mean=-2.1877917989063186e-06, rank=4 mean=5.802757866609229e-07
#can compare mean of residual... the lower the mean the closer it is to the dataset
#but what is the baseline for the noise? what is a good number?
#could plot the means and see when there is a sharp decrease? 

#options: mean, corcoeff, autocorrelation,??


for col in range((np.shape(X))[0]):
    slice2=ddA[:,col]
    R=abs(np.corrcoef(Y,slice2))[0,1]
    #listofR.append(R)
    #print(R)
for row in range((np.shape(Y))[0]):
    slice3=ddA[row,:]
    R1=abs(np.corrcoef(X,slice3))[0,1]
    #listofR.append(R)
    #print(R1)

In [ ]:
#horizontal slice
%matplotlib widget
fig, ax = plt.subplots()


def slider(t):
    #finds closest index to that t
    i = np.searchsorted(Y, t)
    #get horizontal slice
    slice1=Znew[i,:]
    [l.remove() for l in ax.lines]
    ax.plot(X,slice1,color="black")
   
    
widgets.interact(slider, t=(-1,1,0.1))


In [ ]:
#VERTICAL SLICE----------
%matplotlib widget
fig, ax = plt.subplots()


def slider(wl):
    #finds closest index to that wl
    i = np.searchsorted(X, 512)
    print(i)
    slice2=Z[:,i];
    [l.remove() for l in ax.lines]
    ax.plot(Y,slice2,color="black")

#Slider used to select wavelength
widgets.interact(slider, wl=(400,700,3))

#need to give it a different amplitude sign
#do the fit with a few initial guesses
#giving it a bad value for pw? so need to find a better thing

